In [ ]:
%reload_kedro

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.metrics import roc_auc_score, classification_report, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import array #useful to parse values

In [ ]:
LABEL_COL = "label"
INDEX_COL = "window_nbr"

In [ ]:
# XGBoost and LSTM
mt_test_multic = catalog.load("master_table_test_multic").set_index(INDEX_COL)
X_test_multic, y_test_multic = mt_test_multic.drop(columns=[LABEL_COL]), mt_test_multic[[LABEL_COL]]

# LogReg
mt_test_nonmultic = catalog.load("master_table_test_nonmultic").set_index(INDEX_COL)
X_test_nonmultic, y_test_nonmultic = mt_test_nonmultic.drop(columns=[LABEL_COL]), mt_test_nonmultic[[LABEL_COL]]

### XGBoost

In [ ]:
model = catalog.load("xgboost_fitted_model")

y_pred = catalog.load("xgboost_model_predict")
df_model_rpt = catalog.load("xgboost_model_reporting")

In [ ]:
_roc_auc_score = roc_auc_score(y_true=y_test_multic, y_score=y_pred)
_roc_auc_score

In [ ]:
print(classification_report(y_true=y_test_multic, y_pred=y_pred))

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test_multic, model.predict_proba(X_test_multic)[:,1])

plt.figure()

plt.plot(fpr, tpr, label='XGBoost (area = %0.2f)' % _roc_auc_score)
plt.plot([0, 1], [0, 1], linestyle="--", color="gray")

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.title('XGBoost ROC Curve')
plt.legend(loc="lower right")

plt.savefig("/Users/gpalazzo/Desktop/fig_xgboost_roc_curve.png")

In [ ]:
cm = eval(df_model_rpt["confusion_matrix"][0])

ax = plt.subplot()
plot = sns.heatmap(cm, annot=True, ax=ax)
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title("XGBoost CM")

fig = plot.get_figure()
fig.savefig("/Users/gpalazzo/Desktop/fig_xgboost_cm.png")

### LSTM

In [ ]:
from crypto_thesis.data_domains.modeling.lstm import _build_lstm_timestamps_seq

In [ ]:
y_pred = catalog.load("lstm_model_predict")
model = catalog.load("lstm_fitted_model")
seq_length = catalog.load("params:lstm_timestamp_seq_length")
df_model_rpt = catalog.load("lstm_model_reporting")

In [ ]:
_roc_auc_score = roc_auc_score(y_true=y_test_multic, y_score=y_pred)
_roc_auc_score

In [ ]:
print(classification_report(y_true=y_test_multic, y_pred=y_pred))

In [ ]:
X_test_lstm, _ = _build_lstm_timestamps_seq(X=X_test_multic, y=y_test_multic, seq_length=seq_length)
M_TEST = X_test_lstm.shape[0]

predict_probas = model.predict(x=X_test_lstm, batch_size=M_TEST, verbose=1)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test_multic, predict_probas.ravel())

plt.figure()
plt.plot(fpr, tpr, label='LSTM (area = %0.2f)' % _roc_auc_score)

plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.title('LSTM ROC Curve')
plt.legend(loc="lower right")

plt.savefig("/Users/gpalazzo/Desktop/fig_lstm_roc_curve.png")

In [ ]:
cm = eval(df_model_rpt["confusion_matrix"][0])

ax = plt.subplot()
plot = sns.heatmap(cm, annot=True, ax=ax)
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title("LSTM CM")

fig = plot.get_figure()
fig.savefig("/Users/gpalazzo/Desktop/fig_lstm_cm.png")

### LogReg

In [ ]:
model = catalog.load("logreg_fitted_model")
y_pred = catalog.load("logreg_model_predict")
df_model_rpt = catalog.load("logreg_model_reporting")

In [ ]:
_roc_auc_score = roc_auc_score(y_true=y_test_nonmultic, y_score=y_pred)
_roc_auc_score

In [ ]:
print(classification_report(y_true=y_test_nonmultic, y_pred=y_pred))

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test_nonmultic, model.predict_proba(X_test_nonmultic)[:,1])

plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % _roc_auc_score)

plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.title('Logistic Regression ROC Curve')
plt.legend(loc="lower right")

plt.savefig("/Users/gpalazzo/Desktop/fig_logreg_roc_curve.png")

In [ ]:
cm = eval(df_model_rpt["confusion_matrix"][0])

ax = plt.subplot()
plot = sns.heatmap(cm, annot=True, ax=ax)
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title("Logistic Regression CM")

fig = plot.get_figure()
fig.savefig("/Users/gpalazzo/Desktop/fig_logreg_cm.png")